# Results Inspection

In [12]:
import pandas as pd
import glob
import os

pd.set_option('display.max_colwidth', 0)

In [13]:
# origloss_NBTale = pd.read_csv("./logs/OrigLoss_V5/NBTale_results_wav2vec2_NO_origLossV5_2023-09-18.csv")
# origloss_NBTale.drop(columns=["Unnamed: 0", "wav_file", "path", "text"], inplace=True)
# origloss_NBTale.rename(columns={"asr_str": "asr_origloss", "wer":"wer_origloss", "asd":"asd_origloss"}, inplace=True)

In [14]:
path = "./logs/OrigLoss_V5/"
all_files = glob.glob(os.path.join(path , "*.csv"))

origloss_dfs = []
for file in all_files:
    df = pd.read_csv(file)
    df.drop(columns=["Unnamed: 0", "wav_file", "path", "text"], inplace=True)
    df.rename(columns={"asr_str": "asr_origloss", "wer":"wer_origloss", "asd":"asd_origloss"}, inplace=True)
    origloss_dfs.append(df)

origloss = pd.concat(origloss_dfs, ignore_index=True)

In [15]:
# customloss_NBTale = pd.read_csv("./logs/CustomLoss_V7/NBTale_results_wav2vec2_NO_customLossV7_2023-10-02.csv")
# customloss_NBTale.drop(columns=["Unnamed: 0", "wav_file", "path", "text", "segment_id"], inplace=True)
# customloss_NBTale.rename(columns={"asr_str": "asr_customloss", "wer":"wer_customloss", "asd":"asd_customloss"}, inplace=True)

In [16]:
path = "./logs/CustomLoss_V7/"
all_files = glob.glob(os.path.join(path , "*.csv"))

customloss_dfs = []
for file in all_files:
    df = pd.read_csv(file)
    df.drop(columns=["Unnamed: 0", "wav_file", "path", "text", "segment_id"], inplace=True)
    df.rename(columns={"asr_str": "asr_customloss", "wer":"wer_customloss", "asd":"asd_customloss"}, inplace=True)
    customloss_dfs.append(df)

customloss = pd.concat(customloss_dfs, ignore_index=True)

### Merged results of orig loss & custom loss

In [17]:
merged_results = pd.merge(origloss, customloss, on="ref_str")
merged_results = merged_results.reindex(columns=["segment_id", "ref_str", "asr_origloss", "wer_origloss", "asd_origloss", "asr_customloss", "wer_customloss", "asd_customloss"])

,segment_id,ref_str,asr_origloss,wer_origloss,asd_origloss,asr_customloss,wer_customloss,asd_customloss
0,N2_030505_NRK_D12_NO_cut_0,kraftselskapet hafslund blir solgt hvis prisen er god nok sier oslo kommune som ikke vil la seg presse av regjeringen,kraftselskapet hafslund blir solgt hvis prisen er god nok sier oslo kommune som ikke vil la seg presse av regjeringen,0.000000,0.000000,kraftselskapet hafslund blir solgt hvis prisen er god nok sier oslo kommune som ikke vil la seg presse av regjeringen,0.000000,0.000000
1,N2_030505_NRK_D12_NO_cut_1,kvinnen som var savnet i et sjøfly i telemark er funnet omkommet og folk sender gaver til maud angelica men,kvinnen som var savnet i et sjøfly i telemark er funnet omkommet og folk sender gaver til mod angelika men,0.100000,0.123614,kvinnen som var savnet i et sjøfly i telemark er funnet omkommet og folk sender gaver til mod angelika men,0.100000,0.123614
2,N2_030505_NRK_D12_NO_cut_2,hva hun får er hemmelig her er nrk dagsnytt klokka er tolv tretti først til striden rundt kraftselskapet hafslund oslo,hva hun får er hemmelig her er n r k dagsnytt klokka er tolv tretti først til striden rundt kraftselskapet hafslund oslo,0.150000,0.136711,hva hun får er hemmelig her er n r k dagsnytt klokka er tolv tretti først til striden rundt kraftselskapet hafslund oslo,0.150000,0.136711
3,N2_030505_NRK_D12_NO_cut_3,kommune vil ikke la seg presse til å beholde aksjene i selskapet olje og energiministeren vil stanse salget for å,kommune vil ikke la seg presse til å beholde aksjene i selskapet olje og energiministeren vil stanse salget for å,0.000000,0.000000,kommune vil ikke la seg presse til å beholde aksjene i selskapet olje og energiministeren vil stanse salget for,0.050000,0.052995
4,N2_030505_NRK_D12_NO_cut_4,sikre norske eiere i hafslund men finansbyråd i oslo andré støylen lar seg ikke presse og sier at selskapet blir,sikre norske eiere i hafslund men finansbyråd i oslo andre støylen lar seg ikke presse og sier at selskapet blir,0.050000,0.070778,sikre norske eiere i hafslund men finansbyråd i oslo andre støylen lar seg ikke presse og sier at selskapet blir,0.050000,0.070778
...,...,...,...,...,...,...,...,...
824,2021H264-fullStorting0510Stortinget-20210510-115459-Sak1_cut_292,økonomiske utviklingen til kommunen ikke var bærekraftig det er sterke ord i en perspektivmelding resultatet av det var at høyre,økonomiske utviklingen til kommunen ikke var bærekraftig og det er sterke ord i en perspektivmelding resultatet av det var at høyre,0.050000,0.061569,økonomiske utviklingen til kommunen ikke var bærekraftig og det er sterke ord i en perspektivmelding resultatet av det var at høyre,0.050000,0.061569
825,2021H264-fullStorting0510Stortinget-20210510-115459-Sak1_cut_293,frp med frp finansbyråd innførte eiendomsskatt det var ikke noe venstre eller krf i det byrådet som tvang dem til,frpbyrådet med frp finansbyråd innførte eiendomsskatt og det var ikke noe venstre eller krf i det byrådet som tvang de til det det,0.250000,0.229273,frpbyrådet med frp finansbyråd innførte eiendomsskatt og det var ikke noe venstre eller krf i det byrådet som tvang de tider,0.200000,0.234647
826,2021H264-fullStorting0510Stortinget-20210510-115459-Sak1_cut_294,det det var frps finansbyråd frps egen kollega her på stortinget silje hjemdal som i dag er representant på stortinget,frp sin finansbyråd og frp sin egen kollega her på stortinget silje hjemdal som i dag er representant her på stortinget,0.350000,0.278696,frp sin finansbyråd og frp sin egen kollega her på stortinget silje hjemdal som i dag er representant her på stortinget,0.350000,0.278696
827,2021H264-fullStorting0510Stortinget-20210510-115459-Sak1_cut_295,satt i dette byrådet så det er bare å forvente at kollegaene hennes går tilbake fra denne saken og unnskylder,satt i dette byrådet så det er bare til å forvente at kollegaene hennes går tilbake fra denne saken unnskylder,0.100000,0.098308,satt i dette byrådet så det er jo bare til å forvente at kollegaene hennes

In [18]:
better_wer = merged_results[merged_results["wer_origloss"] > merged_results["wer_customloss"]]
worse_wer = merged_results[merged_results["wer_origloss"] < merged_results["wer_customloss"]]
better_asd = merged_results[merged_results["asd_origloss"] > merged_results["asd_customloss"]]
worse_asd = merged_results[merged_results["asd_origloss"] < merged_results["asd_customloss"]]

### better WER but *worse* ASD when using custom loss

In [19]:
better_wer[better_wer["asd_origloss"] < better_wer["asd_customloss"]]

,segment_id,ref_str,asr_origloss,wer_origloss,asd_origloss,asr_customloss,wer_customloss,asd_customloss
11,N2_030505_NRK_D12_NO_cut_11,kan gjøre noe her inne og kor dykk eventuelt vil arrangere meg finansbyråden i oslo måtte svare på mange henvendelser,marineonline finansbyråden i oslo måtte svare på mange henvendelser,0.60,0.492595,kan nå hariris finansbyråden i oslo måtte svare på mange henvendelser,0.55,0.521667
272,N2_050211_NRK_D17_NO_OOLremoved_cut_156,han held fast på at han er uskuldig han er naturligvis fortvilet og skuffet over resultatet straffeutmålingen noterer en at,han held fast på at han er uskuldeghan er naturligvis fortvilet og skuffet over resultatet straffutmålingen noterer en at,0.15,0.098264,han held fast på at han er uskyldig han er naturligvis fortvilet og skuffet over resultatet straffutmålingen noterer en at,0.10,0.123088
397,N2_051206_NRK_D12_NO_OOLremoved_cut_45,i og med at dei har mistillit frå ni av tretten medlemmar i landsrådet så det betyr at spelet er,i og med at de har mistillit fra ni av tretten medlemmer i landsrådet så det betyr at spillet er,0.20,0.109594,i og med at dei har mistillit fra ni av tretten medlemmer i landsrådet så det betyr at spillet er,0.15,0.110652
478,p1_g02_m2_5_x-free_cut_13,jo sånn rett ned fem hundre meter utfor autovernet de andre kom rett etter meg og klart så hva altså,sånn rett ned fem hundre meter utføre ut vernet de andre komme rett etter meg og klarte å så kvasse,0.45,0.490793,sånn rett ned fem hundre meter utføre utover de andre kom rett etter meg og klarte å så kasomskjedd,0.35,0.552101
545,2021H264-fullStorting0510Stortinget-20210510-115459-Sak1_cut_13,eigedomsskatt innan tjue tjuefire eigedomsskatten er ein kommunal skatt og høyrer inn under det kommunale sjølvstyret det er opp til,eigendomsskatt innen tjue tjuefire eiendomsskatten er en kommunal skatt og hører inn under det kommunale sjølstyret det er opp til,0.30,0.172710,eigendomskatt innen tjue tjuefire eiendomsskatten er en kommunal skatt og høyrer inn under det kommunale sjølstyret det er opp til,0.25,0.176661
723,2021H264-fullStorting0510Stortinget-20210510-115459-Sak1_cut_191,renovasjonsavgiften har også hatt et betydelig løft for å få dette til man har hatt gateopprustninger der man har fått,innovasjonssystem renovasjonsavgiften har også hatt et betydelig løft for å få dette til og man har hatt gateopprustninger la andre hvordan også har fått,0.30,0.301255,innovasjonssystem renovasjonsavgiften har også hatt et betydelig løft for å få dette til og man har hatt gateopprustninger klandre hordeogså har fått,0.20,0.383861
821,2021H264-fullStorting0510Stortinget-20210510-115459-Sak1_cut_289,gjør dette som et heltidsverv også helgheim og johnsen og johansen og flere fra frp bør unnskylde til egne kollegaer,gjør dette som et heltidsverv og så bør jo helgheim johnsen johansen og flere fra frp unnskylder til egne kolleger,0.45,0.272333,gjør dette som et heltidsverv og så blir jo helge johnsen og johansen og flere fra frp unnskylder til egne kolleger,0.40,0.277996
825,2021H264-fullStorting0510Stortinget-20210510-115459-Sak1_cut_293,frp med frp finansbyråd innførte eiendomsskatt det var ikke noe venstre eller krf i det byrådet som tvang dem til,frpbyrådet med frp finansbyråd innførte eiendomsskatt og det var ikke noe venstre eller krf i det byrådet som tvang de til det det,0.25,0.229273,frpbyrådet med frp finansbyråd innførte eiendomsskatt og det var ikke noe venstre eller krf i det byrådet som tvang de tider,0.20,0.234647


### worse WER but *better* ASD when using custom loss

In [20]:
worse_wer[worse_wer["asd_origloss"] > worse_wer["asd_customloss"]]

,segment_id,ref_str,asr_origloss,wer_origloss,asd_origloss,asr_customloss,wer_customloss,asd_customloss
9,N2_030505_NRK_D12_NO_cut_9,å sikre et nasjonalt eigarskap det er en debatt som dei nasjonale politikarane får ta eg får forholde meg til,sikre et nasjonalt eierskap det er en debatt som dei nasjonale politikerne får ta for forholde meg til,0.25,0.284005,sikre et nasjonalt eierskap det er en debatt som de nasjonale politikerne får ta jeg for forholde meg til,0.30,0.245522
97,N2_030505_NRK_D12_NO_cut_97,vinglet frem og tilbake i dette spørsmålet og fikk til slutt beskjed av statsminister guy verhofstadt om å godta nattflyvninger,vinglet frem og tilbake i dette spørsmålet og fikk til slutt beskjed av statsminister gifer hofstad om å godta nattflygningr,0.15,0.282468,vinglet frem og tilbake i dette spørsmålet og fikk til slutt beskjed av statsminister gifer hovstad om å godta natt flyvninger,0.20,0.197169
121,N2_050211_NRK_D17_NO_OOLremoved_cut_5,liksom sånn masse forskjellige oster og masse forskjellig melkesort altså ost rømme hva det nå helst måtte være så vil,liksom sånn masse forskjellige oster og masse forskjellig melketår altså ut være,0.50,0.488312,liksom sånn masse forskjellige oster og masse forskjellig melke stort altså ottere,0.60,0.478594
265,N2_050211_NRK_D17_NO_OOLremoved_cut_149,ut i frå det tapet dei føle då tingrettsdommar terje mowatt i dag leste opp drapsdommen på tretten år satt,få det tapet deg føle då tingrett dommer terje mowat i dag leste opp drapsdommen på tretten år satt,0.35,0.314504,ut ifrå det tapet deg føle da tingrett dommer terje mowat i dag leste opp drapsdommen på tretten år sa,0.40,0.239040
302,N2_050211_NRK_D17_NO_OOLremoved_cut_186,kvart å mista lyttarar og dermed inntekter til kanal tjuefire dette er bakgrunnen for at radioselskapa kan bli slått saman,kvart og mista lytter og dermed inntekter til kanal tjuefire dette er bakgrunnen for at radio selskapet kan bli slått sammen,0.25,0.233716,kvart og mistet lyttere og dermed inntekter til kanal tjuefire dette er bakgrunnen for at radio selskapet kan bli slått sammen,0.30,0.220201
402,N2_051206_NRK_D12_NO_OOLremoved_cut_50,med ein viss verdigheit men det er framleis usikkert om ho blir sittande eller ikkje og det får vi ikkje,med en viss verdighet men det er framleis usikkert om hun blir sittende eller ikkje og det får vi ikke,0.25,0.223463,med en viss verdighet men det er framleis usikkert om hun blir sittende eller ikke og det får vi ikke,0.30,0.207827
500,p1_g04_m2_5_x-free_cut_8,til spania så dagen etter altså søndag da satt vi allerede på flyet og landa i madrid sein seint på,til spania så dagen etter altså søndag da satt vi allerede på flyet og landet i madrid sin seint på,0.10,0.103505,til spania så dagen etter altså søndag da satt vi allerede på flyet og landet i madrid sent sent på,0.15,0.079296
524,p1_g08_m2_5_x-free_cut_4,av det mest spennende jeg har gjort er å være tilknyttet senter for borgerkrigsforskning som var ett av de såkalte,det mest spennende jeg har gjort er å være tilknyttet senter for borgerkrigsfrskning som var et av de såkalte,0.15,0.188180,det mest spennende jeg har gjort det å være tilknyttet til senter for borgerkrigsforskning som var et av de såkalte,0.20,0.151398
561,2021H264-fullStorting0510Stortinget-20210510-115459-Sak1_cut_29,på om kommunane kan løysa velferdsoppgåvene sine på ein tilfredsstillande måte eller ikkje med denne regjeringen har vi kuttet skatter,på om kommunene kan løse velferdsoppgavene sine på en tilfredsstillende måte eller ikke ja takk neste taler er representanten aleksander stokkebø høyre deretter ørsal johansen stokkebø ver så god president denne regjeringen har me kuttet skatter,1.20,1.092610,på om kommunene kan løyse velferdsoppgavene sine på en tilfredsstillende måte eller ikke tak takk neste taler er representanten aleksander stoke høyre deretter ørsal johansen stokkebø ver så god president med denne regjeringen har me kutte skatter,1.25,1.068461
666,2021H264-fullStorting0510Stortinge

In [21]:
wer_improved_percent = (len(better_wer)/len(merged_results)) * 100
asd_improved_percent = (len(better_asd)/len(merged_results)) * 100

wer_worsened_percent = (len(worse_wer)/len(merged_results)) * 100
asd_worsened_percent = (len(worse_asd)/len(merged_results)) * 100

In [22]:
print(f"WER improved: {wer_improved_percent:.2f}%")
print(f"ASD improved: {asd_improved_percent:.2f}%")

print(f"WER worsened: {wer_worsened_percent:.2f}%")
print(f"ASD worsened: {asd_worsened_percent:.2f}%")

WER improved: 19.30%
ASD improved: 25.33%
WER worsened: 17.13%
ASD worsened: 24.85%


# CTC LOSS IMPLEMENTATION

In [1]:
import collections
if not hasattr(collections, "Container"):
    import collections.abc
    collections.Container = collections.abc.Container
import transformers
from transformers import AutoTokenizer, BertModel
from transformers import Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor, Wav2Vec2Processor
from transformers import Wav2Vec2ForCTC, Wav2Vec2ProcessorWithLM, TrainingArguments, Trainer
from datasets import load_dataset, load_metric, ClassLabel, Audio, Dataset
import random
import pandas as pd
import math
import numpy as np
import librosa
import os
import torch
from pydub import AudioSegment
from IPython.display import display, HTML
import re
import json
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union
import wandb
import argparse
import types
from tabulate import tabulate
from dtw import *
import torch
from scipy.spatial import distance
import copy
import pickle

/localhome/stipendiater/janinelr/miniconda3/envs/wav2vec/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



In [2]:
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\*]'
def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower()
    return batch

def prepare_dataset(batch):
    audio = batch["audio"]
    # batched output is "un-batched" to ensure mapping is correct
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    with processor.as_target_processor():
        batch["labels"] = processor(batch["text"]).input_ids
    return batch

In [3]:
model_name = "NbAiLab/nb-wav2vec2-300m-bokmaal"

processor = Wav2Vec2ProcessorWithLM.from_pretrained(model_name)
processor_woLM = Wav2Vec2Processor.from_pretrained(model_name)

# model = Wav2Vec2ForCTC.from_pretrained(model_name)
model = Wav2Vec2ForCTC.from_pretrained(
    model_name,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
)

/localhome/stipendiater/janinelr/miniconda3/envs/wav2vec/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:97: FutureWarning: Deprecated argument(s) used in 'snapshot_download': allow_regex. Will not be supported from version '0.12'.

Please use `allow_patterns` and `ignore_patterns` instead.
  warnings.warn(message, FutureWarning)
Fetching 4 files: 100%|██████████| 4/4 [00:00<00:00, 15.10it/s]


In [4]:
# sum(p.numel() for p in model.parameters() if p.requires_grad)

In [4]:
df_train = pd.read_csv("../../model_ckpts/fine-tuning_wav2vec2_customLossV11/runs/train_set.csv")
df_train.drop(columns=["Unnamed: 0"], inplace=True)
train_dataset = Dataset.from_pandas(df_train)
train_dataset = train_dataset.cast_column("path", Audio())
train_dataset = train_dataset.rename_column("path", "audio")
train_dataset = train_dataset.cast_column("audio", Audio(sampling_rate=16_000))
train_dataset = train_dataset.map(prepare_dataset, remove_columns=train_dataset.column_names, num_proc=4)

#0:   0%|          | 0/5468 [00:00<?, ?ex/s]/localhome/stipendiater/janinelr/miniconda3/envs/wav2vec/lib/python3.10/site-packages/transformers/models/wav2vec2_with_lm/processing_wav2vec2_with_lm.py:584: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/localhome/stipendiater/janinelr/miniconda3/envs/wav2vec/lib/python3.10/site-packages/transformers/models/wav2vec2_with_lm/processing_wav2vec2_with_lm.py:584: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(

/localhome/stipendiater/janinelr/miniconda3/envs/wav2vec/lib/python3.10/site-packages/transformers/

In [11]:
input_feature = [{"input_values": train_dataset[2]["input_values"]}]
label_feature = [{"input_ids": train_dataset[2]["labels"]}]

batch = processor.pad(input_feature, padding=True, return_tensors="pt")

with processor.as_target_processor():
    label = processor.pad(label_feature, padding=True, return_tensors="pt")
label = label["input_ids"].masked_fill(label.attention_mask.ne(1), -100)

batch["labels"] = label

In [12]:
batch

{'input_values': tensor([[-0.4508, -0.3754, -0.2800,  ...,  0.2719,  0.2864,  0.3113]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]], dtype=torch.int32), 'labels': tensor([[23, 15, 26, 14,  9,  1,  3, 11,  9, 19,  0,  4, 18, 29, 13,  0, 15, 13,
          0, 19,  5,  9,  5, 18,  0, 11, 14, 21, 19, 20,  0,  1, 22,  0, 18, 21,
         19, 19,  9, 19, 11,  5,  0, 10,  5, 11,  1, 20,  5, 18,  9, 14,  1,  0]])}

### Stanford CTC LOSS

In [3]:
def ctc_loss(params, seq, blank=0):
    seqLen = seq.shape[0]  # length of label sequence
    numphones = params.shape[0]  # number of labels, not even used in the code
    L = 2*seqLen + 1  # length of the label sequence with blanks
    T = params.shape[1]  # length of utterance (time)

    # print(seqLen, numphones, L, T)

    alphas = np.zeros((L,T))
    betas = np.zeros((L,T))

    # convert logits to log softmax
    params = params - np.max(params,axis=0)
    params = np.exp(params)
    params = params / np.sum(params,axis=0)

    # initialize alphas and forward pass
    alphas[0,0] = params[blank,0]
    alphas[1,0] = params[seq[0],0]
    c = np.sum(alphas[:,0])
    alphas[:,0] = alphas[:,0] / c
    llForward = np.log(c)

    for t in range(1,T):
        start = max(0,L-2*(T-t))
        end = min(2*t+2,L)
        for s in range(start,L):
            l = int((s-1)/2)
            # blank
            if s%2 == 0:
                if s==0:
                    alphas[s,t] = alphas[s,t-1] * params[blank,t]
                else:
                    alphas[s,t] = (alphas[s,t-1] + alphas[s-1,t-1]) * params[blank,t]
            # same label twice
            elif s == 1 or seq[l] == seq[l-1]:
                alphas[s,t] = (alphas[s,t-1] + alphas[s-1,t-1]) * params[seq[l],t]
            else:
                alphas[s,t] = (alphas[s,t-1] + alphas[s-1,t-1] + alphas[s-2,t-1]) * params[seq[l],t]

        # normalize at current time (prevent underflow)
        c = np.sum(alphas[start:end,t])
        alphas[start:end,t] = alphas[start:end,t] / c
        llForward += np.log(c)

    # initialize betas and backwards pass
    betas[-1,-1] = params[blank,-1]
    betas[-2,-1] = params[seq[-1],-1]
    c = np.sum(betas[:,-1])
    betas[:,-1] = betas[:,-1] / c
    llBackward = np.log(c)

    for t in range(T-2,-1,-1):
        start = max(0,L-2*(T-t))
        end = min(2*t+2,L)
        for s in range(end-1,-1,-1):
            l = int((s-1)/2)
            # blank
            if s%2 == 0:
                if s == L-1:
                    betas[s,t] = betas[s,t+1] * params[blank,t]
                else:
                    betas[s,t] = (betas[s,t+1] + betas[s+1,t+1]) * params[blank,t]
            # same label twice
            elif s == L-2 or seq[l] == seq[l+1]:
                betas[s,t] = (betas[s,t+1] + betas[s+1,t+1]) * params[seq[l],t]
            else:
                betas[s,t] = (betas[s,t+1] + betas[s+1,t+1] + betas[s+2,t+1]) * params[seq[l],t]

        # normalize at current time
        c = np.sum(betas[start:end,t])
        betas[start:end,t] = betas[start:end,t] / c
        llBackward += np.log(c)

    # Compute gradient with respect to unnormalized input parameters
    grad = np.zeros(params.shape)
    ab = alphas*betas
    for s in range(L):
        l = int((s-1)/2)
        # blank
        if s%2 == 0:
            grad[blank,:] += ab[s,:]
            ab[s,:] = ab[s,:]/params[blank,:]
        else:
            grad[seq[l],:] += ab[s,:]
            ab[s,:] = ab[s,:]/(params[seq[l],:])
    absum = np.sum(ab,axis=0)

    # Check for underflow or zeros in denominator of gradient
    llDiff = np.abs(llForward-llBackward)
    if llDiff > 1e-5 or np.sum(absum==0) > 0:
        print("Diff in forward/backward LL : %f"%llDiff)
        print("Zeros found : (%d/%d)"%(np.sum(absum==0),absum.shape[0]))
        # print(-llForward, grad)
        return (-llForward, grad)
    else:
        grad = params - grad / (params * absum)
        # print(-llForward, grad)
        return (-llForward, grad)

In [2]:
def ctc_loss_tensor(params, seq, blank=0):
    seqLen = seq.shape[0]  # length of label sequence
    L = 2*seqLen + 1  # length of the label sequence with blanks
    T = params.shape[1]  # length of utterance (time)

    alphas = torch.zeros((L,T))
    betas = torch.zeros((L,T))

    # convert logits to log probs
    params = params - (torch.max(params, dim=0)[0])
    params = torch.exp(params)
    params = params / torch.sum(params, dim=0)

    # initialize alphas and forward pass
    alphas[0,0] = params[blank,0]
    alphas[1,0] = params[seq[0],0]
    c = torch.sum(alphas[:,0])
    alphas[:,0] = alphas[:,0] / c
    llForward = torch.log(c)

    for t in range(1,T):
        start = max(0,L-2*(T-t))
        end = min(2*t+2,L)
        for s in range(start,L):
            l = int((s-1)/2)
            # blank
            if s%2 == 0:
                if s==0:
                    alphas[s,t] = alphas[s,t-1] * params[blank,t]
                else:
                    alphas[s,t] = (alphas[s,t-1] + alphas[s-1,t-1]) * params[blank,t]
            # same label twice
            elif s == 1 or seq[l] == seq[l-1]:
                alphas[s,t] = (alphas[s,t-1] + alphas[s-1,t-1]) * params[seq[l],t]
            else:
                alphas[s,t] = (alphas[s,t-1] + alphas[s-1,t-1] + alphas[s-2,t-1]) * params[seq[l],t]

        # normalize at current time (prevent underflow)
        c = torch.sum(alphas[start:end,t])
        alphas[start:end,t] = alphas[start:end,t] / c
        llForward = llForward + torch.log(c)

    # initialize betas and backwards pass
    betas[-1,-1] = params[blank,-1]
    betas[-2,-1] = params[seq[-1],-1]
    c = torch.sum(betas[:,-1])
    betas[:,-1] = betas[:,-1] / c
    llBackward = torch.log(c)

    for t in range(T-2,-1,-1):
        start = max(0,L-2*(T-t))
        end = min(2*t+2,L)
        for s in range(end-1,-1,-1):
            l = int((s-1)/2)
            # blank
            if s%2 == 0:
                if s == L-1:
                    betas[s,t] = betas[s,t+1] * params[blank,t]
                else:
                    betas[s,t] = (betas[s,t+1] + betas[s+1,t+1]) * params[blank,t]
            # same label twice
            elif s == L-2 or seq[l] == seq[l+1]:
                betas[s,t] = (betas[s,t+1] + betas[s+1,t+1]) * params[seq[l],t]
            else:
                betas[s,t] = (betas[s,t+1] + betas[s+1,t+1] + betas[s+2,t+1]) * params[seq[l],t]

        # normalize at current time
        c = torch.sum(betas[start:end,t])
        betas[start:end,t] = betas[start:end,t] / c
        llBackward = llBackward + torch.log(c)

    # Compute gradient with respect to unnormalized input parameters
    grad = torch.zeros(params.shape)
    ab = alphas*betas
    for s in range(L):
        l = int((s-1)/2)
        # blank
        if s%2 == 0:
            grad[blank,:] = grad[blank,:] + ab[s,:]
            ab[s,:] = ab[s,:]/params[blank,:]
        else:
            grad[seq[l],:] = grad[seq[l],:] + ab[s,:]
            ab[s,:] = ab[s,:]/(params[seq[l],:])
    absum = torch.sum(ab,axis=0)

    # Check for underflow or zeros in denominator of gradient
    llDiff = torch.abs(llForward-llBackward)
    if llDiff > 1e-5 or torch.sum(absum==0) > 0:
        print("Diff in forward/backward LL : %f"%llDiff)
        print("Zeros found : (%d/%d)"%(torch.sum(absum==0),absum.shape[0]))
        # print(-llForward, grad)
        return (-llForward, grad)
    else:
        grad = params - grad / (params * absum)
        # print(-llForward, grad)
        return (-llForward, grad)

### Training Trial

In [25]:
model.train()
output = model(**batch)

In [26]:
logits = output["logits"][0].detach().cpu().numpy().transpose()
seq = batch["labels"][0].detach().cpu().numpy()

loss, grad = ctc_loss(logits, seq.squeeze(), blank=0)
print(loss)

428.9810518359343


In [27]:
import torch.nn.functional as F

logits = output["logits"][0]
log_probs = F.log_softmax(logits, dim=-1, dtype=torch.float32)
targets = batch["labels"][0]
input_lengths = torch.tensor(output["logits"][0].shape[0])
target_lengths = torch.tensor(batch["labels"][0].shape[0])

loss = F.ctc_loss(log_probs=log_probs, targets=targets, input_lengths=input_lengths, target_lengths=target_lengths, reduction="sum")
print(loss)

tensor(428.9811, grad_fn=<SumBackward0>)


In [60]:
# torch.autograd.set_detect_anomaly(True)

logits = output["logits"][0].transpose(1,0)
seq = batch["labels"][0]

loss, grad = ctc_loss_tensor(logits, seq, blank=0)
print(loss)
# loss.backward()

# optim = torch.optim.AdamW(model.parameters(), lr=1e-3)
# optim.step()

Diff in forward/backward LL : 0.000031
Zeros found : (0/203)
tensor(428.9811, grad_fn=<NegBackward0>)


In [29]:
logits

tensor([[  1.4962,   1.0071,   1.6117,  ...,   2.2399,   6.2250,   9.0228],
        [ -2.6267,  -2.2761,  -0.2974,  ...,   0.2028,   0.0742,  -1.0642],
        [ -4.1935,  -3.8417,  -3.9895,  ...,  -3.0990,  -2.9767,  -2.8504],
        ...,
        [  5.8301,   4.6000,   3.2805,  ...,   6.2615,   3.8786,   3.0810],
        [-15.2999, -15.3380, -14.9798,  ..., -15.0261, -14.9139, -14.6649],
        [-15.4351, -15.3723, -15.0139,  ..., -15.0612, -15.0430, -14.8176]],
       grad_fn=<TransposeBackward0>)

# CTC Loss & ASD alignment

In [36]:
logits = output["logits"][0]
label_ids = batch["labels"][0]

pred_ids = np.argmax(logits.detach().numpy(), axis=-1)
logit_frames_decoded = processor.tokenizer.batch_decode(pred_ids, group_tokens=False)
print(logit_frames_decoded)

label_ids = batch["labels"][0]
label_ids[label_ids == -100] = processor.tokenizer.pad_token_id
print(processor.tokenizer.batch_decode(label_ids, group_tokens=False))

['[PAD]', 'n', 'n', 'e', '', '', 'v', 'a', 'a', '[PAD]', '[PAD]', 's', 's', '[PAD]', 'n', '[PAD]', '[PAD]', 'j', 'a', 'a', 'a', 'k', 'k', '[PAD]', '[PAD]', 'k', '[PAD]', '[PAD]', 'e', '[PAD]', '[PAD]', '[PAD]', 's', '[PAD]', '[PAD]', 'e', '[PAD]', '[PAD]', '[PAD]', '', '', 'd', 'd', 'r', 'r', 'r', 'ø', 'ø', 'm', 'm', '[PAD]', '[PAD]', '[PAD]', '', '', '', 'o', 'm', 'm', '[PAD]', '[PAD]', '[PAD]', '', '', '', '[PAD]', '[PAD]', 's', '[PAD]', '[PAD]', '[PAD]', 'e', 'e', 'i', 'i', 'i', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', 'e', 'e', 'r', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '', '', '', '[PAD]', 'k', 'k', '[PAD]', '[PAD]', '[PAD]', '[PAD]', 'n', 'n', 'u', 'u', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', 's', '[PAD]', '[PAD]', '[PAD]', '', '', 'a', 'a', 'a', '[PAD]', 'v', 'v', '[PAD]', '', 'r', 'r', 'u', 'u', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', 't', 't', '[PAD]', 'i', '[PAD]', '[PAD]', '[PAD]', '[PAD]', 's', 's', 's', '[PA

In [21]:
# pickle.dump(logit_frames_decoded, open("logit_frames.pkl", "wb"))

In [37]:
ref = processor_woLM.batch_decode(batch["labels"], group_tokens=False)[0]
output_logits = output["logits"].detach().numpy()
hyp = processor.batch_decode(output_logits).text[0]

print("REF:", ref)
print("HYP:", hyp)

REF: wozniackis drøm om seier knust av russiske jekaterina
HYP: asnake drøm om sier knut av russiske karine


In [32]:
# pickle.dump(logits, open("logits_8Nov.pkl", "wb"))
# pickle.dump(logit_frames_decoded, open("logit_frames_decoded_8Nov.pkl", "wb"))

### NOTE: start running here! (when loading from pickle)

In [4]:
# NOV-8 DATA
ref = "wozniackis drøm om seier knust av russiske jekaterina"
hyp = "asnake drøm om sier knut av russiske karine"

logit_frames_decoded = pickle.load(open("logit_frames_decoded_8Nov.pkl", "rb"))

alignment_table = [logit_frames_decoded]
table = tabulate(alignment_table, tablefmt="html")
print("logits decoded")
display(HTML(table))

logits decoded


[PAD],n,n,e,,,v,a,a,[PAD],[PAD],s,s,[PAD],n,[PAD],[PAD],j,a,a,a,k,k,[PAD],[PAD],k,[PAD],[PAD],e,[PAD],[PAD],[PAD],s,[PAD],[PAD],e,[PAD],[PAD],[PAD],,,d,d,r,r,r,ø,ø,m,m,[PAD],[PAD],[PAD],,,,o,m,m,[PAD],[PAD],[PAD],,,,[PAD],[PAD],s,[PAD],[PAD],[PAD],e,e,i,i,i,[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],e,e,r,[PAD],[PAD],[PAD],[PAD],[PAD],,,,[PAD],k,k,[PAD],[PAD],[PAD],[PAD],n,n,u,u,[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],s,[PAD],[PAD],[PAD],,,a,a,a,[PAD],v,v,[PAD],,r,r,u,u,[PAD],[PAD],[PAD],[PAD],[PAD],t,t,[PAD],i,[PAD],[PAD],[PAD],[PAD],s,s,s,[PAD],[PAD],k,[PAD],e,e,e,[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],,,,g,j,e,e,[PAD],[PAD],[PAD],k,k,a,a,a,[PAD],[PAD],[PAD],a,[PAD],[PAD],r,r,[PAD],i,i,[PAD],[PAD],[PAD],[PAD],n,n,[PAD],e,e,[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],,


In [5]:
modelname = 'ltg/norbert2'
model = BertModel.from_pretrained(modelname)
tokenizer = AutoTokenizer.from_pretrained(modelname)

Some weights of the model checkpoint at ltg/norbert2 were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
def get_asd_align(ref, hyp, model, tokenizer):
    tokenized_ref = tokenizer(ref, padding=True, truncation=True, max_length=512, return_tensors="pt")
    tokenized_hyp = tokenizer(hyp, padding=True, truncation=True, max_length=512, return_tensors="pt")
    ref_input_ids = tokenized_ref["input_ids"].squeeze()
    hyp_input_ids = tokenized_hyp["input_ids"].squeeze()

    with torch.no_grad():
        model_output_ref = model(**tokenized_ref, output_hidden_states=True)
        model_output_hyp = model(**tokenized_hyp, output_hidden_states=True)
    hidden_states_ref = model_output_ref.hidden_states
    hidden_states_hyp = model_output_hyp.hidden_states
    all_layers_reference = []
    all_layers_hypothesis = []
    for i in range(1,13):
        all_layers_reference.append(hidden_states_ref[i].squeeze())
        all_layers_hypothesis.append(hidden_states_hyp[i].squeeze())
    ref_embedding_sequence = torch.stack(all_layers_reference).mean(dim=0)
    hyp_embedding_sequence = torch.stack(all_layers_hypothesis).mean(dim=0)

    alignment = dtw(hyp_embedding_sequence, ref_embedding_sequence, dist_method=distance.cosine, keep_internals=True)
    num_tokens = len(ref_embedding_sequence)
    asd_score = (alignment.distance / num_tokens)

    ref_alignment_idxs = alignment.index2
    hyp_alignment_idxs = alignment.index1

    cosdist_alignment_tokens = []  # FOR PRINTING
    # ref_alignments = []
    hyp_alignments = []
    for i in range(len(ref_alignment_idxs)):
        ref_embedding = ref_embedding_sequence[ref_alignment_idxs[i]]
        hyp_embedding = hyp_embedding_sequence[hyp_alignment_idxs[i]]
        cosdist = distance.cosine(ref_embedding, hyp_embedding)
        cosdist_alignment_tokens.append(round(cosdist, 3))  # FOR PRINTING
        # ref_token = tokenizer.convert_ids_to_tokens(ref_input_ids[ref_alignment_idxs[i]].reshape(1))[0]
        hyp_token = tokenizer.convert_ids_to_tokens(hyp_input_ids[hyp_alignment_idxs[i]].reshape(1))[0]
        # ref_alignments.append((ref_token, cosdist))
        hyp_alignments.append((hyp_alignment_idxs[i], hyp_token, cosdist))

    # FOR PRINTING
    ref_alignment_input_ids = np.empty(len(ref_alignment_idxs), dtype=int)
    for i, index in enumerate(ref_alignment_idxs):
        ref_alignment_input_ids[i] = (ref_input_ids[index])
    ref_alignment_tokens = tokenizer.convert_ids_to_tokens(torch.from_numpy(ref_alignment_input_ids))

    hyp_alignment_input_ids = np.empty(len(hyp_alignment_idxs), dtype=int)
    for i, index in enumerate(hyp_alignment_idxs):
        hyp_alignment_input_ids[i] = (hyp_input_ids[index])
    hyp_alignment_tokens = tokenizer.convert_ids_to_tokens(torch.from_numpy(hyp_alignment_input_ids))

    hyp_alignment_tokens.insert(0, "HYP:")
    ref_alignment_tokens.insert(0, "REF:")
    cosdist_alignment_tokens.insert(0, "CosDist:")
    alignment_table = [ref_alignment_tokens, hyp_alignment_tokens, cosdist_alignment_tokens]
    table = tabulate(alignment_table, tablefmt="html")

    print("Token alignment table:")
    display(HTML(table))

    return hyp_alignments

In [7]:
hyp_alignments = get_asd_align(ref, hyp, model, tokenizer)

Token alignment table:


REF:,[CLS],[UNK],[UNK],[UNK],[UNK],drøm,om,seier,knust,knust,av,russiske,je,##kat,##eri,##na,[SEP]
HYP:,[CLS],[CLS],as,##na,##ke,drøm,om,sier,kn,##ut,av,russiske,russiske,kar,kar,##ine,[SEP]
CosDist:,0.19,0.611,0.631,0.712,0.568,0.124,0.179,0.413,0.631,0.753,0.169,0.105,0.593,0.573,0.621,0.48,0.013


In [8]:
# collapse repetitions in tokens and wordpieces in the HYP alignment from ASD
clean_hyp_alignment = []
collapsed_indices = []
for i, item in enumerate(hyp_alignments):
    if i < (len(hyp_alignments) - 1):
        if item[0] == hyp_alignments[i+1][0]:
            averaged_cosdist = sum([item[2], hyp_alignments[i-1][2]]) / 2
            clean_hyp_alignment.append(("collapsed", item[1], averaged_cosdist))
            collapsed_indices.append(item[0])
        elif item[0] not in collapsed_indices and item[1] != "[CLS]":
            clean_hyp_alignment.append(item)


# GROUPING THE TOKENS FROM ASD CALCULATION, SUCH THAT WORDPIECES ARE TOGETHER
regrouped_tokens = []
for i, item in enumerate(clean_hyp_alignment):
    if item[1] != "[CLS]" and item[1] != "[SEP]":
        if "##" not in item[1] and "##" in clean_hyp_alignment[i+1][1]:  # start of a group of wordpieces
            wordpiece_group = []
            wordpiece_group.append(item)
            regrouped_tokens.append(wordpiece_group)
        elif "##" in item[1]:  # parts of the word
            wordpiece_group.append(item)
        else:  # not wordpieces
            regrouped_tokens.append(item)


# for token in regrouped_tokens:
#     print(token)


# COLLAPSE WORDPIECES INTO WORDS & TAKE AVERAGE OF COSDIST
tokens_compressed = []
for token_group in regrouped_tokens:
    if isinstance(token_group, list):
        wp_combined = ''.join([wordpiece[1].replace("##", "") for wordpiece in token_group])
        token_ave_cosdist = sum([wordpiece[2] for wordpiece in token_group]) / len(token_group)
        tokens_compressed.append(("combined", wp_combined, token_ave_cosdist))
    else:
        tokens_compressed.append(token_group)


for token in tokens_compressed:
    print(token)

('combined', 'asnake', 0.6369385619958242)
(4, 'drøm', 0.12406861782073975)
(5, 'om', 0.17880988121032715)
(6, 'sier', 0.4131525754928589)
('combined', 'knut', 0.6922986581921577)
(9, 'av', 0.16893887519836426)
('collapsed', 'russiske', 0.13685202598571777)
('combined', 'karine', 0.531245581805706)


In [9]:
# GROUPING CHARACTERS FROM LOGITS w/ FRAME NOS. & DISREGARDING ALL THE PADS

clean_logit_frames = []
for i, item in enumerate(logit_frames_decoded):
    if item != "[PAD]":
        clean_logit_frames.append((i, item))

char_group_list = []
for i, char in enumerate(clean_logit_frames):
    if i == 0:
        if char[1] == "":
            continue
    else:
        if len(char_group_list) == 0:  # first word
            if char[1] != "":
                new_word = []
                new_word.append(char)
                char_group_list.append(new_word)
        else:
            if clean_logit_frames[i-1][1] == "" and char[1] != "":  # start of the word
                new_word = []
                new_word.append(char)
                char_group_list.append(new_word)
            elif char[1] != "":
                new_word.append(char)
            else:
                continue

for item in char_group_list:
    print(" ".join([char[1] for char in item]))
    print(item)



n e
[(2, 'n'), (3, 'e')]
v a a s s n j a a a k k k e s e
[(6, 'v'), (7, 'a'), (8, 'a'), (11, 's'), (12, 's'), (14, 'n'), (17, 'j'), (18, 'a'), (19, 'a'), (20, 'a'), (21, 'k'), (22, 'k'), (25, 'k'), (28, 'e'), (32, 's'), (35, 'e')]
d d r r r ø ø m m
[(41, 'd'), (42, 'd'), (43, 'r'), (44, 'r'), (45, 'r'), (46, 'ø'), (47, 'ø'), (48, 'm'), (49, 'm')]
o m m
[(56, 'o'), (57, 'm'), (58, 'm')]
s e e i i i e e r
[(67, 's'), (71, 'e'), (72, 'e'), (73, 'i'), (74, 'i'), (75, 'i'), (83, 'e'), (84, 'e'), (85, 'r')]
k k n n u u s
[(95, 'k'), (96, 'k'), (101, 'n'), (102, 'n'), (103, 'u'), (104, 'u'), (113, 's')]
a a a v v
[(119, 'a'), (120, 'a'), (121, 'a'), (123, 'v'), (124, 'v')]
r r u u t t i s s s k e e e
[(127, 'r'), (128, 'r'), (129, 'u'), (130, 'u'), (136, 't'), (137, 't'), (139, 'i'), (144, 's'), (145, 's'), (146, 's'), (149, 'k'), (151, 'e'), (152, 'e'), (153, 'e')]
g j e e k k a a a a r r i i n n e e
[(163, 'g'), (164, 'j'), (165, 'e'), (166, 'e'), (170, 'k'), (171, 'k'), (172, 'a'), (173, '

In [13]:
# GET THE ALIGNMENT BETWEEN ASD TOKENS/WORDS & LOGIT FRAMES


In [ ]:
# DIVIDE THE ASD COSDIST PER NO. OF FRAMES BASED ON ALIGNMENT & ASIGN AVERAGE COSDIST PER FRAME


In [ ]:
# DIVIDING THE ASD COSDIST PER NO. FRAMES
if len(char_group_list) == len(regrouped_tokens):
    ASD_cosdist_list = []
    for i, char_group in enumerate(char_group_list):
        # char_group_copy = copy.deepcopy(char_group)
        if isinstance(regrouped_tokens[i], list): # token character list differs when there are whole word tokens or wordpieces
            token_char_list = list(''.join([wordpiece[1].replace("##", "") for wordpiece in regrouped_tokens[i]]))
            token_cosdist = sum([wordpiece[2] for wordpiece in regrouped_tokens[i]])
        else:
            token_char_list = list(regrouped_tokens[i][1])
            token_cosdist = regrouped_tokens[i][2]

        # simpler implementation by not checking if each frame belongs to the decoded word
        # it probably does not need checking anyway since we're taking the whole sequence when decoding
        cosdist_divided = token_cosdist / len(char_group)  # divide ASD token cosdist by number of frames for word utterance
        frame_cosdist = []
        for char in char_group:
            frame_cosdist.append((char[0], char[1], cosdist_divided))
        ASD_cosdist_list.append(frame_cosdist)
else:
    print("logit char grouped frames quantity not equal to regrouped tokens from ASD")

In [ ]:
for item in ASD_cosdist_list:
    print(' '.join([char[1] for char in item]))

In [ ]:
for item in ASD_cosdist_list:
    print(item)

### other scripts:

In [ ]:
# GROUPING CHARACTERS FROM LOGITS MATRIX INTO THEIR CORRESPONDING WORDS, WHILE RETAINING ROW NOS. OR FRAME NOS.

char_group_list = []
for i, char in enumerate(logit_frames_decoded):
    if i == 0:
        if char == "[PAD]" or char == "":
            continue
    else:
        if char == "[PAD]" or char == "":
            continue
        if len(char_group_list) == 0:  # first word
            if logit_frames_decoded[i-1] in ["", "[PAD]"] and char not in ["", "[PAD]"]:
                new_word = []
                new_word.append((i, char))
                char_group_list.append(new_word)
        else:
            if "" in logit_frames_decoded[i-2:i-1] and char not in ["", "[PAD]"]:  # start of the word
                new_word = []
                new_word.append((i, char))
                char_group_list.append(new_word)
            elif logit_frames_decoded[i-1] == char and char != "":  # repeating characters
                new_word.append((i, char))
            elif logit_frames_decoded[i-1] != char and char != "":  # new character detected
                new_word.append((i, char))


for item in char_group_list:
    print(' '.join([char[1] for char in item]))

In [ ]:
if len(char_group_list) == len(regrouped_tokens):
    ASD_cosdist_list = []
    for i, char_group in enumerate(char_group_list):
        # char_group_copy = copy.deepcopy(char_group)
        if isinstance(regrouped_tokens[i], list): # token character list differs when there are whole word tokens or wordpieces
            token_char_list = list(''.join([wordpiece[1].replace("##", "") for wordpiece in regrouped_tokens[i]]))
            token_cosdist = sum([wordpiece[2] for wordpiece in regrouped_tokens[i]])
        else:
            token_char_list = list(regrouped_tokens[i][1])
            token_cosdist = regrouped_tokens[i][2]

        # detecting applicable frames & caclulating average cosdist
        # frame_group = []
        # for token_char in token_char_list:
        #     detect_count = 0
        #     if token_char in [x[1] for x in char_group_copy]:
        #         for i, char in enumerate(char_group_copy):
        #             # print(char[1], token_char, [x[1] for x in char_group_copy])
        #             if char[1] == token_char:
        #                 frame_group.append(char)
        #                 detect_count += 1
        #             else:
        #                 if detect_count > 0:
        #                     for count in range(detect_count):
        #                         char_group_copy.pop(0)
        #                     break
        #     else:
        #         continue
        # cosdist_divided = token_cosdist / len(frame_group)  # divide ASD token cosdist by number of frames for word
        # assign divided ASD token cosdist
        # frame_cosdist = []
        # for char in frame_group:
        #     if char[1] in token_char_list:
        #         frame_cosdist.append((char[0], char[1], cosdist_divided))

        # simpler implementation by not checking if each frame belongs to the decoded word
        # it probably does not need checking anyway since we're taking the whole sequence when decoding
        cosdist_divided = token_cosdist / len(char_group)  # divide ASD token cosdist by number of frames for word utterance
        frame_cosdist = []
        for char in char_group:
            frame_cosdist.append((char[0], char[1], cosdist_divided))
        ASD_cosdist_list.append(frame_cosdist)
else:
    print("logit char grouped frames quantity not equal to regrouped tokens from ASD")